<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import os
import numpy as np
from scipy import stats
from scipy.stats.mstats import gmean  # For geometric mean
from statsmodels.stats.multitest import multipletests # for fdr correction

In [2]:
# File path
filepath = "/Users/nehamurad/Desktop/Ph1/Ph1_Epistem/counts/dosedvsplacebo/"

In [3]:
# Define cohorts, contrasts, and prefixes for file names
cohorts = ["A", "B", "D", "F", "C", "E", "G"]
contrasts = ["Day_3_48hr_vs_placebo", "Day_8__vs_placebo"]
# prefixes = ["combined_cohort_", "IVcombined_cohort_", "SCcombined_cohort_", "IV_cohort_", "SC_cohort_"]
prefixes = ["IVcombined_cohort_", "SCcombined_cohort_"]

# Set significance threshold
alpha = 0.05

In [4]:
# Create an empty list to store all DataFrames
all_results = []

# Loop over cohorts, contrasts, and prefixes
for cohort in cohorts:
    for contrast in contrasts:
        for prefix in prefixes:
            filename = f"{filepath}{prefix}{cohort}_{contrast}.csv"
            if os.path.exists(filename):
                df = pd.read_csv(filename, index_col=0)

                # Add cohort and contrast columns to the DataFrame
                df["cohort"] = cohort
                df["contrast"] = contrast
                df["prefix"] = prefix
                
                df = df.dropna(subset=['pvalue'])

                all_results.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(all_results)

# Group by gene name and apply the meta-analysis
combined_results = merged_df.groupby(merged_df.index).agg({
    "pvalue": lambda x: stats.combine_pvalues(x, method='fisher')[1],  # Combine pvalues values directly
    "log2FoldChange": lambda x: np.log2(gmean(2**x))  # Calculate geometric mean on fold changes
})

# Rename columns and count significant results per gene
combined_results = combined_results.rename(columns={"pvalue": "combined_pvalue", "log2FoldChange": "combined_log2FoldChange"})
combined_results["significance_count"] = merged_df.groupby(merged_df.index)["pvalue"].apply(lambda x: (x < alpha).sum())
combined_results["combined_padj"] = multipletests(combined_results["combined_pvalue"], method='fdr_bh')[1]
combined_results["-log10(pvalue_adj)"] = -np.log10(combined_results["combined_padj"])

# Sort the results (no need to adjust again)
combined_results = combined_results.sort_values(
    by=["significance_count", "combined_pvalue"], ascending=[False, True]
)

fp = "/Users/nehamurad/Desktop/Ph1/Ph1_Epistem/counts/metanalysis_results/"

fn = "dosed_placebo_all.csv"

combined_results.to_csv(fp+fn)

In [5]:
combined_results.head()

,combined_pvalue,combined_log2FoldChange,significance_count,combined_padj,-log10(pvalue_adj)
S100A4,2.287552e-11,0.279801,8,2.017621e-08,7.695161
KCNN3,4.093746e-10,-0.681475,8,1.378625e-07,6.860554
HAND2,8.416320e-10,1.830699,8,2.398263e-07,6.620103
ENTPD8,3.927725e-09,-0.261501,8,7.202904e-07,6.142492
KLHDC7A,4.614354e-09,0.455715,8,8.117840e-07,6.090559


In [7]:
combined_results[(combined_results.significance_count>5) & (combined_results.combined_padj<0.05)]

,combined_pvalue,combined_log2FoldChange,significance_count,combined_padj,-log10(pvalue_adj)
S100A4,2.287552e-11,0.279801,8,2.017621e-08,7.695161
KCNN3,4.093746e-10,-0.681475,8,1.378625e-07,6.860554
HAND2,8.416320e-10,1.830699,8,2.398263e-07,6.620103
ENTPD8,3.927725e-09,-0.261501,8,7.202904e-07,6.142492
KLHDC7A,4.614354e-09,0.455715,8,8.117840e-07,6.090559
...,...,...,...,...,...
TSKU-AS1,2.562362e-04,0.126861,6,2.420307e-03,2.616130
KRTAP2-3,3.472832e-04,-0.059517,6,3.045634e-03,2.516322
PCSK6,3.989330e-04,-0.043821,6,3.373990e-03,2.471856
H1-4,5.540450e-04,-0.544816,6,4.355697e-03,2.360942
